# Step 1: Project Mining - GitHub

In [26]:
import os
import datetime
import random
import json
try: 
    from github import Github
    import pickle
except ImportError:
    print("Lib github not local available")
    import pip
    cmd = "pip install PyGithub"
    os.system(cmd)
    cmd = "pip install pickle"
    os.system(cmd)
    import pickle
CONFIG_FILES_PATH = "/home/configFiles/GitHubProjects/"

In [2]:
from github import Github

g = Github("maes95","<token>")

query="""
    language:java 
    stars:>=500 
    forks:>=300 
    created:<2015-01-01 
    pushed:>2020-01-01
    archived:false
    is:public
"""
repositories=[]

repositories = g.search_repositories(query=query)

In [3]:
repos= []
count=0

for repo in repositories:
    repos.append(repo)
    count=count+1

In [4]:
date=str(datetime.datetime.now())

with open('repos_%s.pickle'%date, 'wb') as f:
    pickle.dump(repos, f)

print("Total repos: %d"%len(repos))

Total repos: 682


In [5]:
file=""
# 'repos_%s.pickle'%date
with open('repos_%s.pickle'%date, 'rb') as f:
    repos = pickle.load(f)
filtered_repos = []

for repo in repos:
    print(repo.full_name)
    contents = repo.get_contents("")
    for content_file in contents:
        if content_file.path in ["build.gradle", "pom.xml", "build.xml"]:
            filtered_repos.append(repo)
            break
            
print("Total repos: %d"%len(filtered_repos))

682
iluwatar/java-design-patterns
elastic/elasticsearch
spring-projects/spring-boot
ReactiveX/RxJava
spring-projects/spring-framework
google/guava
square/okhttp
square/retrofit
apache/dubbo
PhilJay/MPAndroidChart
bumptech/glide
zxing/zxing
JakeWharton/butterknife
netty/netty
skylot/jadx
greenrobot/EventBus
alibaba/fastjson
alibaba/druid
eugenp/tutorials
Netflix/Hystrix
ReactiveX/RxAndroid
SeleniumHQ/selenium
square/picasso
libgdx/libgdx
chrisbanes/PhotoView
nostra13/Android-Universal-Image-Loader
apache/kafka
google/ExoPlayer
jenkinsci/jenkins
bazelbuild/bazel
alibaba/canal
android10/Android-CleanArchitecture
DrKLO/Telegram
mybatis/mybatis-3
dianping/cat
redisson/redisson
looly/hutool
apache/flink
signalapp/Signal-Android
brettwooldridge/HikariCP
hdodenhof/CircleImageView
EnterpriseQualityCoding/FizzBuzzEnterpriseEdition
openzipkin/zipkin
winterbe/java8-tutorial
LMAX-Exchange/disruptor
greenrobot/greenDAO
Bigkoo/Android-PickerView
daimajia/AndroidSwipeLayout
eclipse/deeplearning4j
daim

spring-projects/spring-integration-samples
gitblit/gitblit
dlazaro66/QRCodeReaderView
RoaringBitmap/RoaringBitmap
DeemOpen/zkui
apache/activemq
SalomonBrys/ANR-WatchDog
joel-costigliola/assertj-core
JabRef/jabref
hamcrest/JavaHamcrest
internetarchive/heritrix3
notnoop/java-apns
jcoleman/tomcat-redis-session-manager
medcl/elasticsearch-analysis-pinyin
docker-java/docker-java
oshi/oshi
DozerMapper/dozer
apache/incubator-gobblin
sarxos/webcam-capture
elastic/elasticsearch-hadoop
redsolution/xabber-android
OryxProject/oryx
hierynomus/sshj
deeplearning4j/nd4j
ukanth/afwall
shopizer-ecommerce/shopizer
Trinea/android-auto-scroll-view-pager
huaban/jieba-analysis
logstash/logstash-logback-encoder
kiegroup/optaplanner
drewnoakes/metadata-extractor
apache/avro
robovm/robovm
vaadin/framework
jmxtrans/jmxtrans
moquette-io/moquette
VoltDB/voltdb
spring-projects/spring-batch
pinterest/secor
bytedeco/javacpp-presets
normanmaurer/netty-in-action
spring-cloud/spring-cloud-config
EngineHub/WorldEdit
life

In [6]:
MAX_COMMITS = 10000
MIN_COMMITS = 1000
filtered_repos_2 = []
for repo in filtered_repos:
    commits = repo.get_commits().totalCount
    if commits >= MIN_COMMITS and commits <= MAX_COMMITS:
        filtered_repos_2.append(repo)
print("Total projects %d"%len(filtered_repos_2))

Total projects 298


In [ ]:
#filtered_repos_3 = [r for r in filtered_repos_2 if r.full_name.split("/")[0] != "apache"]
#print("Total projects %d"%len(filtered_repos_3))

In [7]:
filtered_repos_3 = []
for repo in filtered_repos_2:
    contents = repo.get_contents("")
    isAndroid = False
    for file in contents:
        if file.path == "build.gradle":
            isAndroid = 'com.android.tools.build' in str(repo.get_contents("build.gradle").decoded_content)
            break
    if not isAndroid: filtered_repos_3.append(repo)
print("Total projects %d"%len(filtered_repos_3))

Total projects 252


In [10]:
# SELECCIONAR 30 PROYECTOS
sampling = random.choices(filtered_repos_3, k=40)
len(sampling)

40

In [ ]:
for project in sampling:
    project_name = project.full_name.split("/")[1]
    print(project.full_name)

    config={
        'project': project_name,
        'git_url': project.clone_url,
        'last_commit': project.get_commits()[0].sha[0:8],
        'experiment': 1
    }

    with open('%s/%s-config.json'%(CONFIG_FILES_PATH, project_name), 'w') as outfile:
        json.dump(config, outfile, indent=4)

In [ ]:
for f in os.listdir(CONFIG_FILES_PATH):
    path = os.path.join(CONFIG_FILES_PATH, f)
    if os.path.isfile(path):
        with open(path, 'r') as f:
            config_file = json.load(f)
            
            project_name = config_file['project']
            folder  = "/home/projects/%s" % project_name
            git_url = config_file['git_url']

            # CHECK IF PROJECT EXISTS
            if os.path.exists(folder):
                print(" -> Project exist in local folder!")
            else:
                print(" -> Project %s cloned!"%project_name)
                !git clone $git_url $folder > /dev/null 2>&1
            